In [2]:
import glob
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from datetime import timedelta

In [3]:
def simple_read(path):
    '''
    Reads .ict files to a Pandas DataFrame
    :param path: path to the .ict data
    :return: Pandas DataFrame with .ict data
    '''
    with open(path) as f:
        # find the value in the file which tells you how many lines to skip to get to the table
        first_line = f.readline()
        header_line = int(first_line[0:-2].split(",")[0])-1
    data = pd.read_csv(path, sep=',', skiprows=header_line)

    # finds the location in the path containing the date
    acc = 0
    boo = False
    for letter in path:
        if letter == '2':
            boo = True
        elif boo and letter == '0':
            acc -= 1
            break
        acc += 1
        
    # creates datetime object with the date the data was collected
    dt = datetime(int(path[acc:acc+4]), int(path[acc+4:acc+6]), int(path[acc+6:acc+8])) 
    
    for column in data.keys():
        if 'Time' in column:
            # converts seconds after midnight columns to datetime
            data[column] = dt + pd.to_timedelta(data[column], unit='seconds')
    data.columns = data.columns.str.replace(' ', '')
    return data.replace(-9999, np.nan) # Converts -9999 values to NaN


def add_leg(data, legs):
    '''
    add leg to the data file
    :param data: pandas data
    :param legs: pandas legs data
    :return: Pandas DataFrame with legs
    '''
    data=data.copy()
    # creates leg column
    data['leg'] = np.nan

    # leg codes corresponding to each leg type
    leg_key = {'00':'Takeoff/Landing', '01':'Transit', '02':'BCB', '03':'ACB', '04':'BCT', '05':'ACT', '06':'MinAlt', '07':'Ascent', '08':'Descent', '09':'Slant/Spiral', '10':'BBL', '11':'ABL', '12':'Remote Sensing (HSRL-2)', '13':'Other'}

    data = data.astype({'leg': 'str'})
    for i in range(0, len(legs)):
        subset = data[(data['Time_Mid'] >= legs.iloc[i]['Time_Start']) & (data['Time_Mid'] <= legs.iloc[i]['Time_Stop'])].copy()
        subset['leg'] = leg_key[str(legs.iloc[i]['LegIndex'])[-2:]]
        data.loc[subset.index, 'leg'] = subset['leg']

    return data

In [4]:
# reads each file type
# all data is saved in the data folder within the github repository
ccn_paths = sorted(glob.glob('../data/*CCN*'))
master_ccn = []
for i in range(0, len(ccn_paths)):
    master_ccn.append(simple_read(ccn_paths[i]))
master_ccn = pd.concat(master_ccn).reset_index()
ams_paths = sorted(glob.glob('../data/*AMS_*'))
master_ams = []
for i in range(0, len(ams_paths)):
    master_ams.append(simple_read(ams_paths[i]))
master_ams = pd.concat(master_ams).reset_index()
las_paths = sorted(glob.glob('../data/*LAS*'))
master_las = []
for i in range(0, len(las_paths)):
    master_las.append(simple_read(las_paths[i]))
master_las = pd.concat(master_las).reset_index()
smps_paths = sorted(glob.glob('../data/*SMPS*'))
master_smps = []
for i in range(0, len(smps_paths)):
    master_smps.append(simple_read(smps_paths[i]))
master_smps = pd.concat(master_smps).reset_index()
sum_paths = sorted(glob.glob('../data/*SUMMARY*'))
master_sum = []
for i in range(0, len(sum_paths)):
    master_sum.append(simple_read(sum_paths[i]))
master_sum = pd.concat(master_sum).reset_index().rename(columns={'Time_mid':'Time_Mid'})
leg_paths = sorted(glob.glob('../data/*LegFlags*'))
master_leg = []
for i in range(0, len(leg_paths)):
    master_leg.append(simple_read(leg_paths[i]))
master_leg = pd.concat(master_leg).reset_index()
master_smps = add_leg(master_smps, master_leg)
fcdp_paths = sorted(glob.glob('../data/*FCDP*'))
master_fcdp = []
for i in range(0, len(fcdp_paths)):
    master_fcdp.append(simple_read(fcdp_paths[i]))
master_fcdp = pd.concat(master_fcdp).reset_index()
master_fcdp = master_fcdp[['Time_Start', 'LWC_FCDP']]
v_paths = sorted(glob.glob('../data/*2DS-V*'))
master_v = []
for i in range(0, len(v_paths)):
    master_v.append(simple_read(v_paths[i]))
master_v = pd.concat(master_v).reset_index()
master_2ds = master_v[['Time_Start', 'ED-liquid_2DS', 'Ice_Flag_2DS']]
co_paths = sorted(glob.glob('../data/*CO_*'))
master_co = []
for i in range(0, len(co_paths)):
    master_co.append(simple_read(co_paths[i]))
master_co = pd.concat(master_co).reset_index()
master_co = master_co[['Time_Start', 'Time_Stop', 'CO_ppm']]
opt_paths = sorted(glob.glob('../data/*OPT*'))
master_opt = []
for i in range(0, len(opt_paths)):
    master_opt.append(simple_read(opt_paths[i]))
master_opt = pd.concat(master_opt).reset_index()
master_opt = master_opt[['Time_Mid', 'Abs470_total', 'Abs532_total', 'Abs660_total', 'fRH550_RH20to80']]

In [9]:
(master_smps['Time_Stop'] - master_smps['Time_Start']).describe()

count              34025
mean     0 days 00:00:45
std      0 days 00:00:00
min      0 days 00:00:45
25%      0 days 00:00:45
50%      0 days 00:00:45
75%      0 days 00:00:45
max      0 days 00:00:45
dtype: object

In [8]:
(master_ams['Time_Stop'] - master_ams['Time_Start']).describe()

count                        61660
mean     0 days 00:00:30.151589361
std      0 days 00:00:21.905740466
min                0 days 00:00:01
25%                0 days 00:00:30
50%                0 days 00:00:30
75%                0 days 00:00:30
max                0 days 01:06:42
dtype: object

In [10]:
(master_ams['Time_Stop'] - master_ams['Time_Start']).describe()

count                        61660
mean     0 days 00:00:30.151589361
std      0 days 00:00:21.905740466
min                0 days 00:00:01
25%                0 days 00:00:30
50%                0 days 00:00:30
75%                0 days 00:00:30
max                0 days 01:06:42
dtype: object

In [12]:
master_las

,index,Time_Start,nLAS_AmmSO4,sLAS_AmmSO4,vLAS_AmmSO4,LAS_Bin01,LAS_Bin02,LAS_Bin03,LAS_Bin04,LAS_Bin05,...,LAS_Bin18,LAS_Bin19,LAS_Bin20,LAS_Bin21,LAS_Bin22,LAS_Bin23,LAS_Bin24,LAS_Bin25,LAS_Bin26,stdPT
0,0,2020-02-14 17:01:23,598.10,67.11,3.45,1572.25,1248.64,1603.69,1607.85,1448.25,...,0.0,0.0,0.00,0.0,9.5,0.0,0.0,0.00,0.0,NaN
1,1,2020-02-14 17:01:24,553.94,58.04,2.35,1178.67,1339.59,1204.55,1908.08,1237.91,...,0.0,0.0,15.85,0.0,0.0,0.0,0.0,0.00,0.0,NaN
2,2,2020-02-14 17:01:25,574.95,85.87,12.37,1422.15,1405.24,1342.45,1394.79,1110.14,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,11.92,0.0,NaN
3,3,2020-02-14 17:01:26,633.67,64.52,2.41,1878.85,1311.80,1418.41,1721.92,1241.27,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,NaN
4,4,2020-02-14 17:01:27,538.38,60.86,2.52,1297.79,1382.06,1252.74,989.98,1381.20,...,24.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064828,11898,2022-06-18 15:23:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.103
2064829,11899,2022-06-18 15:23:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.104
2064830,11900,2022-06-18 15:23:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.105
2064831,11901,2022-06-18 15:23:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.104


In [11]:
(master_las['Time_Stop'] - master_las['Time_Start']).describe()

KeyError: 'Time_Stop'

In [10]:
def process_row(i):
    # finds the mean/median/max using the SMPS start and end times (because that has the lowest time resolution)
    row = master_smps.iloc[i]
    t_start = row['Time_Start']
    t_stop = row['Time_Stop']
    ccn_row = master_ccn[(master_ccn['Time_mid'] >= t_start) & (master_ccn['Time_mid'] <= t_stop)].median(numeric_only=True)
    las_row = master_las[(master_las['Time_Start'] >= t_start) & (master_las['Time_Start'] <= t_stop)].median(numeric_only=True)
    ams_row = master_ams[(master_ams['Time_Start'] >= t_start-timedelta(seconds=5)) & (master_ams['Time_Stop'] <= t_stop+timedelta(seconds=5))].mean(numeric_only=True)
    sum_row = master_sum[(master_sum['Time_Mid'] >= t_start) & (master_sum['Time_Mid'] <= t_stop)].median(numeric_only=True)
    fcdp_row = master_fcdp[(master_fcdp['Time_Start'] >= t_start) & (master_fcdp['Time_Start'] <= t_stop)].max(numeric_only=True)
    v_row = master_2ds[(master_2ds['Time_Start'] >= t_start) & (master_2ds['Time_Start'] <= t_stop)].max(numeric_only=True)
    co_row = master_co[(master_co['Time_Start'] >= t_start) & (master_co['Time_Stop'] <= t_stop)].median(numeric_only=True)
    opt_row = master_opt[(master_opt['Time_Mid'] >= t_start) & (master_opt['Time_Mid'] <= t_stop)].median(numeric_only=True)
    return ccn_row, las_row, ams_row, sum_row, fcdp_row, v_row, co_row, opt_row

results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in tqdm(range(0, len(master_smps))))

ccn_mean, las_mean, ams_mean, sum_mean, fcdp_mean, v_mean, co_mean, opt_mean = zip(*results)
ccn_mean = pd.DataFrame(list(ccn_mean))
las_mean = pd.DataFrame(list(las_mean))
ams_mean = pd.DataFrame(list(ams_mean))
sum_mean = pd.DataFrame(list(sum_mean))
fcdp_mean = pd.DataFrame(list(fcdp_mean))
v_mean = pd.DataFrame(list(v_mean))
co_mean = pd.DataFrame(list(co_mean))
opt_mean = pd.DataFrame(list(opt_mean))

100%|██████████| 34025/34025 [17:19<00:00, 32.72it/s]


In [17]:
# Merging specified columns into the merged DataFrame
merged = pd.merge(master_smps[['Time_Mid', 'SMPS_Bin01', 'SMPS_Bin02', 'SMPS_Bin03', 'SMPS_Bin04',
       'SMPS_Bin05', 'SMPS_Bin06', 'SMPS_Bin07', 'SMPS_Bin08', 'SMPS_Bin09',
       'SMPS_Bin10', 'SMPS_Bin11', 'SMPS_Bin12', 'SMPS_Bin13', 'SMPS_Bin14',
       'SMPS_Bin15', 'SMPS_Bin16', 'SMPS_Bin17', 'SMPS_Bin18', 'SMPS_Bin19',
       'SMPS_Bin20', 'SMPS_Bin21', 'SMPS_Bin22', 'SMPS_Bin23', 'SMPS_Bin24',
       'SMPS_Bin25', 'SMPS_Bin26', 'SMPS_Bin27', 'SMPS_Bin28', 'SMPS_Bin29',
       'SMPS_Bin30' , 'nSMPS', 'leg']], ams_mean[['Org_Ave_IsoK_STP', 'SO4_Ave_IsoK_STP',
       'NO3_Ave_IsoK_STP', 'NH4_Ave_IsoK_STP', 'Chl_Ave_IsoK_STP',
       'mz42_Ave_IsoK_STP', 'mz43_Ave_IsoK_STP', 'mz44_Ave_IsoK_STP',
       'mz55_Ave_IsoK_STP', 'mz57_Ave_IsoK_STP', 'mz58_Ave_IsoK_STP',
       'mz60_Ave_IsoK_STP', 'mz79_Ave_IsoK_STP', 'mz91_Ave_IsoK_STP']], left_index=True, right_index=True)
merged = pd.merge(merged, ccn_mean[['CCN_SS', 'N_CCN_stdPT']], left_index=True, right_index=True)
merged = pd.merge(merged, las_mean[['LAS_Bin01', 'LAS_Bin02', 'LAS_Bin03', 'LAS_Bin04', 'LAS_Bin05',
       'LAS_Bin06', 'LAS_Bin07', 'LAS_Bin08', 'LAS_Bin09', 'LAS_Bin10',
       'LAS_Bin11', 'LAS_Bin12', 'LAS_Bin13', 'LAS_Bin14', 'LAS_Bin15',
       'LAS_Bin16', 'LAS_Bin17', 'LAS_Bin18', 'LAS_Bin19', 'LAS_Bin20',
       'LAS_Bin21', 'LAS_Bin22', 'LAS_Bin23', 'LAS_Bin24', 'LAS_Bin25',
       'LAS_Bin26', 'nLAS_AmmSO4']], left_index=True, right_index=True)
merged = pd.merge(merged, sum_mean[['Latitude', 'Longitude', 'GPS_altitude',
       'Pressure_Altitude', 'Pitch', 'Roll', 'True_Heading', 'True_Air_Speed',
       'Static_Air_Temp', 'IR_Surf_Temp', 'Static_Pressure', 'Wind_Speed',
       'Wind_Direction']], left_index=True, right_index=True)
merged = pd.merge(merged, fcdp_mean[['LWC_FCDP']], left_index=True, right_index=True)
merged = pd.merge(merged, v_mean[['ED-liquid_2DS', 'Ice_Flag_2DS']], left_index=True, right_index=True)
merged = pd.merge(merged, co_mean[['CO_ppm']], left_index=True, right_index=True)
merged = pd.merge(merged, opt_mean[['Abs470_total', 'Abs532_total', 'Abs660_total', 'fRH550_RH20to80']], left_index=True, right_index=True)

In [18]:
# save the data into a table
merged.to_csv('../tables/2024_03_11_merged.csv', index=False)